### **KEY GENERATION**

In [435]:
def hex_to_bin(h: str) -> str:
    b = ''
    for i in h:
        b += bin(int(i, base=16)).removeprefix('0b').zfill(4)
    return b

In [457]:
display_bin(hex_to_bin('ca5adec66541fddf4f1c201153d8b447d926fda870b8da566965dba4bc91ee34'))

11001010 01011010 11011110 11000110 01100101 01000001 11111101 11011111 01001111 00011100 00100000 00010001 01010011 11011000 10110100 01000111 11011001 00100110 11111101 10101000 01110000 10111000 11011010 01010110 01101001 01100101 11011011 10100100 10111100 10010001 11101110 00110100 


In [436]:
def display_bin(b: str, n: int = 8):
    if len(b) % n != 0:
        b = b.zfill(n * ((len(b) // n) + 1))
    for i in range(0, len(b), n):
        print(b[i:i+n], end=' ')
    print()

**We start with a 64-bit key.**

In [437]:
hex_key = '133457799BBCDFF1'

In [438]:
bin_key = hex_to_bin(hex_key)

In [439]:
display_bin(bin_key)

00010011 00110100 01010111 01111001 10011011 10111100 11011111 11110001 


**P-Box 1**

### **Permutation box**

Permuation boxes don't add any new information.<br>
They just shuffle the already existing bits.<br>
<br>
In the first step, we want to permutate the<br>
64-bits key to form a 56-bit key. This is thus<br>
called a compression P-box.<br>
<br>
len(pb1) is 56. Thus, the index values represent<br>
the position of the bit in the permutated key.<br>
The values in the list represent the position of<br>
the bit in the original key.<br>

In [440]:
pb1 = [
    57, 49, 41, 33, 25, 17,  9,
     1, 58, 50, 42, 34, 26, 18,
    10,  2, 59, 51, 43, 35, 27,
    19, 11,  3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
     7, 62, 54, 46, 38, 30, 22,
    14,  6, 61, 53, 45, 37, 29,
    21, 13,  5, 28, 20, 12,  4,
]

In [441]:
pkey = ''

for i in pb1:
    pkey += bin_key[i-1]                        # p-box indices start from 1

In [442]:
display_bin(pkey, 7)

1111000 0110011 0010101 0101111 0101010 1011001 1001111 0001111 


In [443]:
c0, d0 = pkey[:28], pkey[28:]

In [444]:
display_bin(c0, 7)
display_bin(d0, 7)

1111000 0110011 0010101 0101111 
0101010 1011001 1001111 0001111 


**Key shift**

In [445]:
def left_shift(b: str, n: int) -> str:
    return b[n:] + b[:n]

In [446]:
left_shifts = [
    1, 1, 2, 2, 2, 2, 2, 2,
    1, 2, 2, 2, 2, 2, 2, 1,
]

In [447]:
c, d = c0, d0

print(f'C0 = ', end='')
display_bin(c, 7)
print(f'D0 = ', end='')
display_bin(d, 7)
print()

keys = []

for i in range(16):
    c = left_shift(c, left_shifts[i])
    d = left_shift(d, left_shifts[i])
    keys.append([c, d])
    print(f'C{i+1} = ', end='')
    display_bin(c, 7)
    print(f'D{i+1} = ', end='')
    display_bin(d, 7)
    print()

C0 = 1111000 0110011 0010101 0101111 
D0 = 0101010 1011001 1001111 0001111 

C1 = 1110000 1100110 0101010 1011111 
D1 = 1010101 0110011 0011110 0011110 

C2 = 1100001 1001100 1010101 0111111 
D2 = 0101010 1100110 0111100 0111101 

C3 = 0000110 0110010 1010101 1111111 
D3 = 0101011 0011001 1110001 1110101 

C4 = 0011001 1001010 1010111 1111100 
D4 = 0101100 1100111 1000111 1010101 

C5 = 1100110 0101010 1011111 1110000 
D5 = 0110011 0011110 0011110 1010101 

C6 = 0011001 0101010 1111111 1000011 
D6 = 1001100 1111000 1111010 1010101 

C7 = 1100101 0101011 1111110 0001100 
D7 = 0110011 1100011 1101010 1010110 

C8 = 0010101 0101111 1111000 0110011 
D8 = 1001111 0001111 0101010 1011001 

C9 = 0101010 1011111 1110000 1100110 
D9 = 0011110 0011110 1010101 0110011 

C10 = 0101010 1111111 1000011 0011001 
D10 = 1111000 1111010 1010101 1001100 

C11 = 0101011 1111110 0001100 1100101 
D11 = 1100011 1101010 1010110 0110011 

C12 = 0101111 1111000 0110011 0010101 
D12 = 0001111 0101010 1011001 100

### **Encryption**

In [448]:
hex_message = '0123456789ABCDEF'
bin_message = hex_to_bin(hex_message)

In [449]:
display_bin(bin_message, 4)

0000 0001 0010 0011 0100 0101 0110 0111 1000 1001 1010 1011 1100 1101 1110 1111 


**Initial permutation**

In [450]:
ip = [
    58, 50, 42, 34, 26, 18, 10,  2,
    60, 52, 44, 36, 28, 20, 12,  4,
    62, 54, 46, 38, 30, 22, 14,  6,
    64, 56, 48, 40, 32, 24, 16,  8,
    57, 49, 41, 33, 25, 17,  9,  1,
    59, 51, 43, 35, 27, 19, 11,  3,
    61, 53, 45, 37, 29, 21, 13,  5,
    63, 55, 47, 39, 31, 23, 15,  7,
]

In [451]:
fp = [
    40,  8, 48, 16, 56, 24, 64, 32,
    39,  7, 47, 15, 55, 23, 63, 31,
    38,  6, 46, 14, 54, 22, 62, 30,
    37,  5, 45, 13, 53, 21, 61, 29,
    36,  4, 44, 12, 52, 20, 60, 28,
    35,  3, 43, 11, 51, 19, 59, 27,
    34,  2, 42, 10, 50, 18, 58, 26,
    33,  1, 41,  9, 49, 17, 57, 25,
]

In [452]:
import numpy as np

arr = np.array([i+1 for i in range(64)], int)

In [453]:
def pbox_operation(arr, pbox) -> np.ndarray:
    p = np.zeros(64, int)
    for i in range(len(arr)):
        p[i] = arr[pbox[i]-1]
    return p

In [454]:
initial = pbox_operation(arr, ip)
final = pbox_operation(initial, fp)

In [455]:
print(initial)

[58 50 42 34 26 18 10  2 60 52 44 36 28 20 12  4 62 54 46 38 30 22 14  6
 64 56 48 40 32 24 16  8 57 49 41 33 25 17  9  1 59 51 43 35 27 19 11  3
 61 53 45 37 29 21 13  5 63 55 47 39 31 23 15  7]


In [456]:
print(final)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
